In [37]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import warnings

### run linear regression on dataframe
def linear_regression(df):
    structs = df.columns.values
    x = np.arange(df.shape[0])
    sum_SSE = 0
    m_dict = {}
    b_dict = {}
    for s in structs:
        y = df.loc[:,s].values
        x= x.reshape(-1, 1)
        y= y.reshape(-1, 1)

        lin_reg = LinearRegression(fit_intercept=True)
        lin_reg.fit(x,y)
        
        m_dict[s] = lin_reg.intercept_
        b_dict[s] = lin_reg.coef_
        
#         ## Plot outputs
#         plt.scatter(x, y,  color='black')
#         plt.plot(x, lin_reg.predict(x), color='blue', linewidth=3)

#         plt.xticks(())
#         plt.yticks(())

#         plt.show()
#         plt.close()

        ## SSE
        diff = y - lin_reg.predict(x)
        sum_SSE += np.sum(np.square(diff))
    return m_dict, b_dict, float(sum_SSE)

'''
read in both data and dictionary files, intercept with columes that describe volume
params: path to both files
param: keyword for filter out area of interest
return: dataframe of patientID, VISCODE, exam date and volumes of all regions in dictionary 
'''
def read_csv(file, dictfile, keyword):
    df = pd.read_csv(file)
    dict_df = pd.read_csv(dictfile)
    #print(df.head())

    #filter out rows only with Volume
    dict_df = dict_df[dict_df["TEXT"].str.contains(keyword, case=False, na=False)]
    STcodes = dict_df['FLDNAME'].values
    #print(dict_df.head())
    #print(STcodes)

    extra_cols = ['RID','VISCODE','EXAMDATE']
    column_needed = np.concatenate([extra_cols, STcodes])
    
    #cross reference df with dict_df
    df = df.loc[:,column_needed]
    return df

In [38]:
##### UCSF Cross Sectional Free Surfer 3T data, this will be the baseline
# df = pd.read_csv('ADNI1_all_data/UCSFFSX51_ADNI1_3T_02_01_16.csv')
df = pd.read_csv('ADNI1_all_data/UCSFFSL_02_01_16.csv') ### longitudinal
#print(df.head())

# dict_df = pd.read_csv('ADNI1_all_data/UCSFFSX51_ADNI1_3T_DICT_11_01_13.csv')
dict_df = pd.read_csv('ADNI1_all_data/UCSFFSL_DICT_11_01_13.csv') ### longitudinal

#filter out rows only with Volume
dict_df = dict_df[dict_df["TEXT"].str.contains("Volume", case=False, na=False)]
STcodes = dict_df['FLDNAME'].values
#print(dict_df.head())
#print(STcodes)

extra_cols = ['RID','VISCODE','EXAMDATE','VERSION']
column_needed = np.concatenate([extra_cols, STcodes])
#print(column_needed)
#cross reference df with dict_df
df = df.loc[:,column_needed]
#drop columns with all NAs
df = df.dropna(axis=1, how='all')

display(df.head())

,RID,VISCODE,EXAMDATE,VERSION,ST101SV,ST102CV,ST103CV,ST104CV,ST105CV,ST106CV,...,ST90CV,ST91CV,ST93CV,ST94CV,ST95CV,ST96SV,ST97CV,ST98CV,ST99CV,ST9SV
0,3,m06,2006-03-13,2009-07-01,1498,2927,1904,3416,2245,4485,...,12330,8211,2236,13139,6003,29544,7065,4253,10022,2808
1,3,m12,2006-09-12,2009-07-01,1519,2935,1826,3275,2271,4148,...,12245,8050,2115,13082,6072,29862,7283,4158,9758,2939
2,3,m24,2007-09-12,2009-07-02,1541,3038,1669,3284,2160,4106,...,11212,7615,2147,12711,6065,32496,6781,3901,9237,2987
3,3,sc,2005-09-01,2009-07-01,1661,3261,1936,3278,2040,4299,...,11836,8550,2362,13566,5892,28288,7079,4046,10097,2863
4,4,m06,2006-05-25,2009-08-24,1285,3021,2110,2638,2787,2136,...,14435,10703,2003,11276,6884,19162,7491,3838,10872,1396


In [39]:
# get patients
patients = np.unique(df['RID'].values)

# atrophy_rate_df = pd.DataFrame(columns=range(136))
atrophy_rate_df = pd.DataFrame(index=patients, columns=range(116))
col_set = False
# print(atrophy_rate_df)

sum_SSE = 0

pd.options.mode.chained_assignment = None  # default='warn'

for p in patients:
    pat_df = df[df['RID'] == p]
    
    ##only take into account of patients with more than 3 rows
    if len(pat_df.index) > 2:
        #convert to datetime, then sort by time
        pat_df['EXAMDATE'] = pd.to_datetime(pat_df['EXAMDATE'], format='%Y-%m-%d')
        pat_df = pat_df.sort_values(by=['EXAMDATE'])        
        pat_df.drop(['RID','VISCODE','VERSION','EXAMDATE'], axis=1, inplace=True)
        
        ## turn volume into log
        pat_df = np.log(pat_df)
        
        #drop columns with all NAs
        pat_df = pat_df.dropna(axis=1, how='all')
        ##filling NA values, forward fill for now, MAY CHANGE!!!!
        pat_df = pat_df.fillna(axis=1, method='ffill')
                
#         if len(pat_df.columns.values) == 135 and col_set == False:
        if len(pat_df.columns.values) == 116 and col_set == False:
            all_cols = pat_df.columns.values
            atrophy_rate_df.columns = all_cols
            col_set = True
        
        ## get slope, intercept, SSE
        slope_dict, inter_dict, SSE = linear_regression(pat_df)
        sum_SSE += SSE
    
        for key, value in slope_dict.items():
            atrophy_rate_df.loc[p, key] = float(value)

# atrophy_rate_df['RID'] = atrophy_rate_df['RID'].astype(int)
# atrophy_rate_df.set_index("RID", inplace=True)
atrophy_rate_df.dropna(axis=0, how='all', inplace=True)  ## drop patients not fitting our data
print("sum SSE: " + str(sum_SSE))
display(atrophy_rate_df)

sum SSE: 7443.829800262682


,ST101SV,ST102CV,ST103CV,ST104CV,ST105CV,ST106CV,ST107CV,ST108CV,ST109CV,ST10CV,...,ST90CV,ST91CV,ST93CV,ST94CV,ST95CV,ST96SV,ST97CV,ST98CV,ST99CV,ST9SV
3,7.37992,8.0502,7.58554,8.11103,7.6583,8.38869,7.48289,8.80012,8.27733,14.4661,...,9.40949,9.05464,7.75338,9.51192,8.68604,10.2452,8.87554,8.33547,9.2314,7.94601
4,7.20462,8.01886,7.69141,7.88935,7.95406,7.74443,7.48863,9.05134,8.16273,14.3299,...,9.53681,9.30503,7.59892,9.34353,8.88325,9.8176,8.94429,8.27022,9.31929,7.18555
5,7.29301,7.98337,7.65383,7.92385,7.74351,8.14214,7.16728,9.03834,8.24911,14.3081,...,9.5997,9.22898,7.68328,9.17719,8.87976,9.67726,8.47043,8.4278,9.28171,7.1936
6,7.14956,7.87556,7.39273,8.02094,7.70457,7.873,7.27003,8.77167,7.97071,14.2218,...,9.51385,8.88851,7.986,9.33669,8.82267,9.76753,8.13813,8.01618,9.17608,6.89753
7,7.11534,7.90609,7.77896,7.99131,7.81459,7.93135,7.58465,8.93355,7.97431,14.1187,...,9.44662,8.98762,7.47058,9.09323,8.59457,9.12064,8.79564,8.31503,9.11438,7.28659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,7.42913,8.30444,7.75651,8.70383,8.13313,8.33165,7.69,9.07211,8.27989,14.4301,...,9.68149,9.5884,7.75294,9.31679,8.8064,9.62809,8.7751,8.46383,9.4985,7.46795
1421,7.18694,8.08574,7.5778,7.89704,7.52374,8.1287,7.43235,8.94332,7.87498,14.1214,...,9.34179,8.96479,7.49509,9.06684,8.71263,9.27886,8.46932,8.39087,9.17957,7.32163
1423,7.39545,8.03275,7.24414,8.12852,7.78988,7.59069,7.32084,8.84971,8.11478,14.3346,...,9.32561,9.42637,7.74085,9.19745,8.87556,9.70868,8.55944,8.25111,9.33631,7.31563
1425,7.23592,7.9424,7.35437,7.89501,7.57999,8.02433,7.39785,8.71107,7.95929,14.1021,...,9.47388,9.20949,7.57785,9.2092,8.66307,8.68598,8.74474,8.10834,9.32195,6.81618


In [40]:
### group patients, take average of each col, then SSE = (difference between avg and patient)^2
#### read the diagnosis from CDR, turn into dict
diag = pd.read_csv("ADNI1_patient_diagnosis.csv").set_index('RID')
diag = diag.astype(float)
# print(diag)

normal = []
AD = []
combined = []

for i in diag.index.values:
    if (diag.loc[i,'m48'] == 0) or (diag.loc[i,'m48'] == 0.5):
        normal = np.append(i, normal)
        combined = np.append(i, combined)
    elif (diag.loc[i,'m48'] == 1) or (diag.loc[i,'m48'] == 2) or (diag.loc[i,'m48'] == 3):
        AD = np.append(i, AD)
        combined = np.append(i, combined)

## get intersection between atrophy table and the diagnosis table for groups
combined = np.intersect1d(combined, atrophy_rate_df.index.values)
normal = np.intersect1d(normal, atrophy_rate_df.index.values)
AD = np.intersect1d(AD, atrophy_rate_df.index.values)

# print(atrophy_rate_df)
combined_df = atrophy_rate_df.loc[combined]
normal_df = atrophy_rate_df.loc[normal]
AD_df = atrophy_rate_df.loc[AD]

print(len(normal))
print(len(AD))
print(len(combined))

445
230
675


In [42]:
#### find mean of each structure, calculate SSE
def structure_SSE(df):
    s_SSE = 0
    for col in df.columns.values:
        arr = df[col]
        mean = np.mean(arr)
        s_SSE += np.sum(np.square(arr - mean))
    return s_SSE

In [45]:
### for real data
combined_SSE = structure_SSE(combined_df)
normal_SSE = structure_SSE(normal_df)
AD_SSE = structure_SSE(AD_df)

print(combined_SSE)
print(normal_SSE)
print(AD_SSE)
print(combined_SSE / (normal_SSE + AD_SSE))

7147.629062949775
4257.9035813421215
2580.5909915877087
1.0452050501353987


In [55]:
#### simulation
SSE_df = pd.DataFrame(columns=['normal', 'AD'])
for i in range(10000):
    simulated_combined = list(combined_df.index.values)
    simulated_AD = np.random.choice(simulated_combined, 230, replace=False)
    simulated_normal = np.setdiff1d(simulated_combined, simulated_AD)

    # print(len(simulated_combined))
    # print(len(simulated_AD))
    # print(len(simulated_normal))
    # print(np.intersect1d(simulated_normal, simulated_AD))

    sim_normal_df = combined_df.loc[simulated_normal]
    sim_AD_df = combined_df.loc[simulated_AD]

    sim_normal_SSE = structure_SSE(sim_normal_df)
    sim_AD_SSE = structure_SSE(sim_AD_df)
    
    SSE_df.loc[SSE_df.shape[0]] = [sim_normal_SSE, sim_AD_SSE]    
SSE_df['combined'] = np.full(shape=10000, fill_value=combined_SSE, dtype=np.float)
SSE_df['test_stats'] = SSE_df['combined'] / (SSE_df['normal'] + SSE_df['AD']) 
SSE_df.to_csv("mass_uni_longitudinal_0913.csv", index=False)